In [13]:
import cv2
import numpy as np

from tkinter import Tk
from tkinter.filedialog import askopenfilename

import matplotlib.pyplot as plt

In [11]:
class Line:
    def __init__(self, x1, y1, x2, y2):
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
    
    def get_coords(self):
        return [(self.x1, self.y1), (self.x2, self.y2)]
    
    def get_equation(self):
        m = (self.y2 - self.y1) / (self.x2 - self.x1)
        b = self.y1 - m * self.x1
        return m, b

In [3]:
# Read image 
Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
file = askopenfilename() # show an "Open" dialog box and return the path to the selected file
image = cv2.imread(file, cv2.IMREAD_COLOR)
# Convert the image to gray-scale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # Creating the kernel with numpy
# kernel2 = np.ones((7, 7), np.float32)/25
 
# # Applying the filter
# filtered = cv2.filter2D(src=image, ddepth=-1, kernel=kernel2)

filtered = cv2.blur(image, (7, 7))

# showing the image
cv2.imwrite("show_blurred.png", filtered)

# Find the edges in the image using canny detector
img = image.copy()
edges = cv2.Canny(filtered, 40, 70)
#edges = gray
# Detect points that form a line
threshold = 50
if False:
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold, minLineLength=10, maxLineGap=250)
    # Draw lines on the image
    try:
        for line in lines:
            try:
                x1, y1, x2, y2 = line[0]
                cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 3)
            except:
                i = 0
    except:
        i = 0
else:
    vert_lines = cv2.HoughLines(edges, 1, np.pi/180, threshold, min_theta=np.pi*7/8, max_theta=np.pi*9/8)
    hor_lines = cv2.HoughLines(edges, 1, np.pi/180, threshold, min_theta=np.pi*7/16, max_theta=np.pi*9/16)
    #lines = cv2.HoughLines(edges, 1, np.pi/180, threshold)
    left = False
    right = False
    try:
        for line in vert_lines:
            try:
                rho, theta = line[0]
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a*rho
                y0 = b*rho
                x1 = int(x0 + 1000*(-b))
                y1 = int(y0 + 1000*(a))
                x2 = int(x0 - 1000*(-b))
                y2 = int(y0 - 1000*(a))
                if (left == True and right == True):
                    break
                if left == False and x1 < image.shape[1]/2:
                    cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 3)
                    left = True     #lines outputted in order of confidence, stop at first one
                if right == False and x1 >= image.shape[1]/2:
                    cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 3)
                    right = True    #lines outputted in order of confidence, stop at first one
            except:
                i = 0
    except:
        i = 0
    
    min_bottom = -1
    best_line = []
    try:
        for line in hor_lines:
            try:
                rho, theta = line[0]
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a*rho
                y0 = b*rho
                x1 = int(x0 + 1000*(-b))
                y1 = int(y0 + 1000*(a))
                x2 = int(x0 - 1000*(-b))
                y2 = int(y0 - 1000*(a))
                # avg = (y1+y2)/2
                # if avg > image.shape[0]/2 and (min_bottom == -1 or min_bottom > avg):
                #     best_line = [x1, y1, x2, y2]
                #     min_bottom = avg
                cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 3)
            except:
                i = 0
    except:
        i = 0
# Show result
cv2.imwrite("show_edges.png", edges)
cv2.imwrite("show_result.png", img)

True

In [30]:
def detect(img_arr, gray_arr, filter, angle1=0, angle2=np.pi, req=0): #req = 0, 1, or 2
    ret_line = None
    
    filtered = cv2.filter2D(src=gray_arr, ddepth=-1, kernel=filter)
    cv2.imwrite("show_blurred_" + str(req) + ".png", filtered)

    # Find the edges in the image using canny detector
    img = img_arr.copy()
    thresh1 = 35
    thresh2 = 60
    if req == 2:
        thresh1 = 30
        thresh2 = 50
    edges = cv2.Canny(filtered, thresh1, thresh2)

    # Detect points that form a line
    threshold = 40
    lines = cv2.HoughLines(edges, 1, np.pi/180, threshold, min_theta=angle1, max_theta=angle2)

    try:
        for line in lines:
            try:
                rho, theta = line[0]
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a*rho
                y0 = b*rho
                x1 = int(x0 + 1000*(-b))
                y1 = int(y0 + 1000*(a))
                x2 = int(x0 - 1000*(-b))
                y2 = int(y0 - 1000*(a))
                if (req == 0) or (req == 1 and x1 < image.shape[1]/3) or (req == 2 and x1 > image.shape[1]*2/3):
                    cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 3)
                    ret_line = Line(x1, y1, x2, y2)
                    break
            except:
                i = 0
    except:
        i = 0
    # Show result
    cv2.imwrite("show_edges_" + str(req) + ".png", edges)
    return img, ret_line

def detect(img_arr, filter, angle1, angle2, req): #req = 0, 1, or 2
    filtered = cv2.filter2D(src=img_arr, ddepth=-1, kernel=filter)
    
    return

In [5]:
kernel0 = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])/72
kernel1 = np.array([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]])/72
kernel2 = np.array([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])/72

In [31]:
# Read image 
Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
file = askopenfilename() # show an "Open" dialog box and return the path to the selected file
image = cv2.imread(file, cv2.IMREAD_COLOR)
# Convert the image to gray-scale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
image, bottom = detect(image, gray, kernel0, angle1=np.pi*7/16, angle2=np.pi*9/16)
image, left = detect(image, gray, kernel1, angle1=np.pi*7/8, angle2=np.pi*9/8, req=1)
image, right = detect(image, gray, kernel2, angle1=np.pi*7/8, angle2=np.pi*9/8, req=2)
cv2.imwrite("show_result.png", image)

True

In [27]:
def in_tube(curr_x, curr_y, bottom, left, right): #assumes x and y are within the boundaries of the image
    m, b = bottom.get_equation()
    if curr_y <= (m * curr_x + b):
        return False
    
    m, b = left.get_equation()
    #if curr_x <= (m * left.x1 + curr_y - left.y1) / m:
    if curr_x <= (curr_y - b) / m:
        return False
    
    m, b = right.get_equation()
    #if curr_x >= (m * right.x1 + curr_y - right.y1) / m:
    if curr_x >= (curr_y - b) / m:
        return False
    
    return True

In [28]:
color = np.zeros(image.shape)
#mask = np.full(image.shape, 0.5)
mask = np.zeros(image.shape)
for i in range(len(mask)):
    for j in range(len(mask[i])):
        if in_tube(i, j, bottom, left, right):
            #mask[i][j] = 0
            mask[i][j] = 1
#plt.imshow(color, cmap='Greys_r', alpha=mask)
plt.imsave("mask.png", mask, cmap='Greys_r')

In [29]:
print(str(bottom.x1) + " " + str(bottom.y1) + " " + str(bottom.x2) + " " + str(bottom.y2))
print(str(left.x1) + " " + str(left.y1) + " " + str(left.x2) + " " + str(left.y2))
print(str(right.x1) + " " + str(right.y1) + " " + str(right.x2) + " " + str(right.y2))

-1033 365 959 531
200 -989 78 1006
649 -972 561 1025


In [7]:
# Read image 
Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
file = askopenfilename() # show an "Open" dialog box and return the path to the selected file
image = cv2.imread(file, cv2.IMREAD_COLOR)
# Convert the image to gray-scale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
### pad this

# Creating the kernel with numpy
kernel2 = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])/72
# kernel2 = np.array([[0], [0], [0], [1], [1], [1], [1], [1], [1], [0], [0], [0]])/10
 
# Applying the filter
filtered = cv2.filter2D(src=image, ddepth=-1, kernel=kernel2)

# filtered = cv2.blur(image, (7, 7))

# showing the image
cv2.imwrite("show_blurred.png", filtered)

# Find the edges in the image using canny detector
img = image.copy()
edges = cv2.Canny(filtered, 40, 70)
#edges = gray
# Detect points that form a line
threshold = 50
hor_lines = cv2.HoughLines(edges, 1, np.pi/180, threshold, min_theta=np.pi*7/16, max_theta=np.pi*9/16)

try:
    for line in hor_lines:
        try:
            rho, theta = line[0]
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a*rho
            y0 = b*rho
            x1 = int(x0 + 1000*(-b))
            y1 = int(y0 + 1000*(a))
            x2 = int(x0 - 1000*(-b))
            y2 = int(y0 - 1000*(a))
            # avg = (y1+y2)/2
            # if avg > image.shape[0]/2 and (min_bottom == -1 or min_bottom > avg):
            #     best_line = [x1, y1, x2, y2]
            #     min_bottom = avg
            cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 3)
            break
        except:
            i = 0
except:
    i = 0
# Show result
cv2.imwrite("show_edges.png", edges)
cv2.imwrite("show_result.png", img)

True

In [8]:
# Read image 
Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
file = askopenfilename() # show an "Open" dialog box and return the path to the selected file
image = cv2.imread(file, cv2.IMREAD_COLOR)
# Convert the image to gray-scale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
### pad this

# Creating the kernel with numpy
kernel2 = np.array([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]])/72
# kernel2 = np.array([[0], [0], [0], [1], [1], [1], [1], [1], [1], [0], [0], [0]])/10
 
# Applying the filter
filtered = cv2.filter2D(src=image, ddepth=-1, kernel=kernel2)

# filtered = cv2.blur(image, (7, 7))

# showing the image
cv2.imwrite("show_blurred.png", filtered)

# Find the edges in the image using canny detector
img = image.copy()
edges = cv2.Canny(filtered, 40, 70)
#edges = gray
# Detect points that form a line
threshold = 50
vert_lines = cv2.HoughLines(edges, 1, np.pi/180, threshold, min_theta=np.pi*7/8, max_theta=np.pi*9/8)

try:
    for line in vert_lines:
        try:
            rho, theta = line[0]
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a*rho
            y0 = b*rho
            x1 = int(x0 + 1000*(-b))
            y1 = int(y0 + 1000*(a))
            x2 = int(x0 - 1000*(-b))
            y2 = int(y0 - 1000*(a))
            if x1 < image.shape[1]/2:
                cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 3)
                break
        except:
            i = 0
except:
    i = 0
# Show result
cv2.imwrite("show_edges.png", edges)
cv2.imwrite("show_result.png", img)

True

In [9]:
# Read image 
Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
file = askopenfilename() # show an "Open" dialog box and return the path to the selected file
image = cv2.imread(file, cv2.IMREAD_COLOR)
# Convert the image to gray-scale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
### pad this

# Creating the kernel with numpy
kernel2 = np.array([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])/72
# kernel2 = np.array([[0], [0], [0], [1], [1], [1], [1], [1], [1], [0], [0], [0]])/10
 
# Applying the filter
filtered = cv2.filter2D(src=image, ddepth=-1, kernel=kernel2)

# filtered = cv2.blur(image, (7, 7))

# showing the image
cv2.imwrite("show_blurred.png", filtered)

# Find the edges in the image using canny detector
img = image.copy()
edges = cv2.Canny(filtered, 40, 70)
#edges = gray
# Detect points that form a line
threshold = 50
vert_lines = cv2.HoughLines(edges, 1, np.pi/180, threshold, min_theta=np.pi*7/8, max_theta=np.pi*9/8)

try:
    for line in vert_lines:
        try:
            rho, theta = line[0]
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a*rho
            y0 = b*rho
            x1 = int(x0 + 1000*(-b))
            y1 = int(y0 + 1000*(a))
            x2 = int(x0 - 1000*(-b))
            y2 = int(y0 - 1000*(a))
            if x1 > image.shape[1]/2:
                cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 3)
                break
        except:
            i = 0
except:
    i = 0
# Show result
cv2.imwrite("show_edges.png", edges)
cv2.imwrite("show_result.png", img)

True